# Compare the projective and lens algorithms

Import the lens and projective mds packages. We will setup a dataset on RP^4 and reduce it to RP^2 using both packages (since the lens space with a Z/2 action is projective space) to see if the Lens MDS algorithm works.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import lens_mds
import projective_mds
from importlib import reload

Loading personal version of PPCA. This may not be consistent with
        the published version


In [2]:
T,n = projective_mds.circleRPn(dim=5)
D = projective_mds.graph_distance_matrix(T,k=5)
Y = projective_mds.initial_guess(T,3)

In [3]:
# Reduce the dimension using pmds
X_P,C_P,_ = projective_mds.pmds(Y,D,verbosity=0)

/opt/anaconda2/envs/stiefel-testing/lib/python3.7/site-packages/autograd/tracer.py:48: RuntimeWarning: invalid value encountered in sqrt
  return f_raw(*args, **kwargs)


No change in S matrix. Stopping iterations


In [4]:
# Check that the optimal rotations match the sign matrix.
omega = lens_mds.g_action_matrix(2,3)
omega_true = np.array([[-1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,-1]])
S = lens_mds.optimal_rotation(Y.T,omega,2)
IP_computed = lens_mds.lens_inner_product(Y.T,omega,S)
IP = np.sign(Y@Y.T)*(Y@Y.T)
np.allclose(IP_computed,IP)

True

The above shows that the inner products computed are correct. The sign matrix may not match because it is very sensitive to near-orthogonal vectors. Next we check that the cost functions match:

In [5]:
W = lens_mds.distance_to_weights(D)
cost = lens_mds.setup_cost(omega,S,D,W)
print(cost(Y.T))
proj_cost = projective_mds.setup_cost(D,np.sign(Y@Y.T))
print(proj_cost(Y.T))

7182.168816562724
7182.168816562724


In [6]:
p=2
M = lens_mds.get_masks(S,p)
cost2 = lens_mds.setup_sum_cost(omega,M,D,W,p)
print(cost2(Y.T))

7182.168816562723


In [7]:
X_L,C_L = lens_mds.lmds(Y,D,2,autograd=True)

Compiling cost function...
Computing gradient of cost function...
Optimizing...
Terminated - min stepsize reached after 41 iterations, 0.59 seconds.

Through 1 iterations:
	Computed cost: 3649.80
	Percent cost difference:  49.18
	Percent Difference in S:  7.86
	Computed cost with new M: 1013.34
Compiling cost function...
Computing gradient of cost function...
Optimizing...
Terminated - min grad norm reached after 116 iterations, 1.54 seconds.

Through 2 iterations:
	Computed cost: 23.12
	Percent cost difference:  99.37
	Percent Difference in S:  0.72
	Computed cost with new M: 2.78
Compiling cost function...
Computing gradient of cost function...
Optimizing...
Terminated - max iterations reached after 13.23 seconds.

Through 3 iterations:
	Computed cost: 0.30
	Percent cost difference:  98.71
	Percent Difference in S:  0.08
	Computed cost with new M: 0.24
Compiling cost function...
Computing gradient of cost function...
Optimizing...
Terminated - min grad norm reached after 426 iteratio

In [8]:
# Check that both methods give same result.
print(cost(X_L.T))
print(cost(X_P.T))
print(np.linalg.norm(X_L-X_P))

4673.929430868076
4673.929432039016
0.0083471006269608


Looks good! Now let's test the analytic gradient:

In [9]:
X_A,C_A = lens_mds.lmds(Y,D,2,autograd=False)

Compiling cost function...
Optimizing...
Terminated - min stepsize reached after 15 iterations, 0.27 seconds.

Through 1 iterations:
	Computed cost: 3650.09
	Percent cost difference:  49.18
	Percent Difference in S:  7.87
	Computed cost with new M: 1018.93
Compiling cost function...
Optimizing...
Terminated - min grad norm reached after 141 iterations, 1.63 seconds.

Through 2 iterations:
	Computed cost: 22.70
	Percent cost difference:  99.38
	Percent Difference in S:  0.72
	Computed cost with new M: 2.72
Compiling cost function...
Optimizing...
Terminated - max iterations reached after 11.33 seconds.

Through 3 iterations:
	Computed cost: 0.29
	Percent cost difference:  98.72
	Percent Difference in S:  0.08
	Computed cost with new M: 0.24
Compiling cost function...
Optimizing...
Terminated - min grad norm reached after 567 iterations, 6.72 seconds.

Through 4 iterations:
	Computed cost: 0.24
	Percent cost difference:  16.61
	Percent Difference in S:  0.00
	Computed cost with new M: 0.

In [10]:
print(cost(X_A.T))
print(np.linalg.norm(X_A-X_P))

4673.92943072877
0.05789480760189832


For fun, we're going to try doing the same with Z/p coefficients for some other choices of p.

In [16]:
reload(lens_mds)
p=4
X_L3,_ = lens_mds.lmds(Y,D,p,autograd=True)
X_A3,_ = lens_mds.lmds(Y,D,p,autograd=False)

Compiling cost function...
Computing gradient of cost function...
Optimizing...
Terminated - min stepsize reached after 70 iterations, 1.81 seconds.

Through 1 iterations:
	Computed cost: 3497.63
	Percent cost difference:  35.00
	Percent Difference in S:  28.77
	Computed cost with new M: 2682.49
Compiling cost function...
Computing gradient of cost function...
Optimizing...
Terminated - min stepsize reached after 77 iterations, 1.92 seconds.

Through 2 iterations:
	Computed cost: 961.82
	Percent cost difference:  72.50
	Percent Difference in S:  20.32
	Computed cost with new M: 1655.19
Compiling cost function...
Computing gradient of cost function...
Optimizing...
Terminated - min stepsize reached after 60 iterations, 1.47 seconds.

Through 3 iterations:
	Computed cost: 610.09
	Percent cost difference:  36.57
	Percent Difference in S:  12.46
	Computed cost with new M: 484.77
Compiling cost function...
Computing gradient of cost function...
Optimizing...
Terminated - min stepsize reache

In [20]:
print(cost(X_A3.T))
print(cost(X_L3.T))
print(np.linalg.norm(X_A3-X_L3))
print(np.linalg.norm(X_A-X_A3))

4673.929520078169
4673.929520135265
3.2092804812472916e-07
7.098955817555078
